In [1]:
import MetaTrader5 as mt5
import time
import pandas as pd
from IPython.display import clear_output



In [2]:
# Nos conectamos con MTD5
if not mt5.initialize():
    print("error en la conexion")
    exit()
else:
    print("conexion exitosa")

conexion exitosa


In [3]:
# Función para enviar órdenes de compra
def buy(symbol, volume):
    try:
        order_type = mt5.ORDER_TYPE_BUY
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": mt5.symbol_info_tick(symbol).ask,
            "sl": 0,
            "tp": 0,
            "deviation": 10,
            "magic": 123456,
            "comment": "Buy order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        return result
    except Exception as buy:
        print(buy)

In [4]:
# Función para enviar órdenes de venta
def sell(symbol, volume):
    try:
        order_type = mt5.ORDER_TYPE_SELL
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": mt5.symbol_info_tick(symbol).bid,
            "sl": 0,
            "tp": 0,
            "deviation": 10,
            "magic": 123456,
            "comment": "Sell order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        return result
    except Exception as sell:
        print(f"Excepcion en la venta de futuros: {sell}")

In [5]:

symbol = "EURUSD"
volume = 0.1  # Ajusta el volumen según tus necesidades

previous_signal = None  # Variable para almacenar el estado anterior


In [ ]:
while True:
    # Obtiene los datos en tiempo real
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 1000)
    if rates is None:
        print("Error al obtener datos.")
        break
    
    # Convierte los datos a un DataFrame
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    data.set_index('time', inplace=True)

    # Calcula las medias móviles
    data['SMA_14'] = data['close'].rolling(window=14).mean()
    data['SMA_50'] = data['close'].rolling(window=50).mean()


    # Extraemos los datos para verificar 
    # data.to_excel("probando.xlsx", engine="openpyxl")

    # Obtiene las últimas medias móviles
    last_sma_14 = data['SMA_14'].iloc[-1]
    last_sma_50 = data['SMA_50'].iloc[-1]

    # imprimimos los datos obtenidos
    print("medida 14", last_sma_14)

    print("Medida 50", last_sma_50)
    # Determina el cruce
    if previous_signal is None:
        previous_signal = 'none'
    
    if last_sma_14 > last_sma_50 and previous_signal != 'buy':
        print("Cruce de compra detectado. Realizando compra...")
        buy(symbol, volume)
        previous_signal = 'buy'

        
    elif last_sma_14 < last_sma_50 and previous_signal != 'sell':
        print("Cruce de venta detectado. Realizando venta...")
        sell(symbol, volume)
        previous_signal = 'sell'

    time.sleep(2)
        # obtenemos el saldo actua
    print("\nEn transaccion")

    # tiempo de espera antes de volver a comprobar
    time.sleep(2)
    clear_output(wait=True)


# Cierra la conexión con MetaTrader 5
mt5.shutdown()


medida 14 1.0834292857142855
Medida 50 1.0833186000000001
